In [1]:
using Distributed

In [2]:
M = 100
angle_array = []
for i = 2π/M:2π/M:2π
    append!(angle_array,i)
end

In [3]:
addprocs(31)

31-element Array{Int64,1}:
  2
  3
  4
  5
  6
  7
  8
  9
 10
 11
 12
 13
 14
  ⋮
 21
 22
 23
 24
 25
 26
 27
 28
 29
 30
 31
 32

In [8]:
using Pkg
Pkg.instantiate()

In [7]:
using Statistics
using BenchmarkTools
using LinearAlgebra
using Distributed
using ProgressMeter
push!(LOAD_PATH, pwd())
using DHC_2DUtils
using FFTW
using MLDatasets
using Images
using Interpolations

In [9]:
L = 8
rand_mat = rand(16,16)
filter_hash = fink_filter_hash(1,L,nx=16,wd=2)
temp = DHC_compute(rand_mat,filter_hash);

In [15]:
using HDF5

In [10]:
using Pkg
Pkg.add("IterTools")

 Resolving package versions...
  Updating `~/.julia/environments/v1.3/Project.toml`
  [c8e1da08] + IterTools v1.3.0
  Updating `~/.julia/environments/v1.3/Manifest.toml`
 [no changes]


In [11]:
using IterTools

In [13]:
train_x, train_y = MNIST.traindata()
test_x, test_y   = MNIST.testdata()

lst_test = Array{Any}(undef, 0)
for i = 1:10000
    push!(lst_test,test_x[:,:,i])
end

In [14]:
@everywhere begin
    using Statistics
    using BenchmarkTools
    using LinearAlgebra
    using Distributed
    using ProgressMeter
    push!(LOAD_PATH, pwd())
    using DHC_2DUtils
    using FFTW
    using MLDatasets
    using Images
    using Interpolations
    
    # filter bank
    filter_hash = fink_filter_hash(1,8,nx=256,wd=2)
    
    function mnist_pad(im; θ=0.0)
        impad = zeros(Float64,128,128)
        impad[78:-1:51,51:78] = im'
        imbig = imresize(impad,(256,256))
        if θ != 0.0
            imrot = imrotate(imbig, θ, axes(imbig), Cubic(Throw(OnGrid())))
            imrot[findall(imrot .!= imrot)] .= 0.0
            return imrot
        end
        return imbig
    end
    
    function mnist_DHC(params)
        θ, x = params
        image    = mnist_pad(x[:,:], θ=θ)
        WST = DHC_compute(image, filter_hash)
        return WST
    end
    
end
 
mnist_DHC_out = @showprogress pmap(mnist_DHC, product(angle_array,lst_test))

h5write("mnist_DHC_test_angles.h5", "main/data", mnist_DHC_out)

┌ Warning: `product(xss...)` is deprecated, use `Iterators.product(xss...)` instead.
│   caller = top-level scope at In[14]:37
└ @ Core In[14]:37
┌ Warning: `product(xss...)` is deprecated, use `Iterators.product(xss...)` instead.
│   caller = top-level scope at In[14]:37
└ @ Core In[14]:37
Progress: 100%|█████████████████████████████████████████| Time: 3:57:19


UndefVarError: UndefVarError: h5write not defined

In [ ]:
temp = hcat(mnist_DHC_out...);

In [ ]:
size(temp)

In [ ]:
1+1

In [23]:
h5write("mnist_DHC_test_angles.h5", "main/data", temp)

MethodError: MethodError: no method matching datatype(::Array{Array{Any,1},2})
Closest candidates are:
  datatype(!Matched::HDF5.Attribute) at /n/home12/saydjari/.julia/packages/HDF5/d0V7K/src/HDF5.jl:1017
  datatype(!Matched::HDF5.Dataset) at /n/home12/saydjari/.julia/packages/HDF5/d0V7K/src/HDF5.jl:1015
  datatype(!Matched::Union{Bool, Float32, Float64, Int16, Int32, Int64, Int8, UInt16, UInt32, UInt64, UInt8, HDF5.Reference}) at /n/home12/saydjari/.julia/packages/HDF5/d0V7K/src/HDF5.jl:1020
  ...

In [ ]:
M = 100
angle_array2 = []
for i = 2π/M-π/M:2π/M:2π
    append!(angle_array2,i)
end

In [ ]:
@everywhere begin
    using Statistics
    using BenchmarkTools
    using LinearAlgebra
    using Distributed
    using ProgressMeter
    push!(LOAD_PATH, pwd())
    using DHC_2DUtils
    using FFTW
    using MLDatasets
    using Images
    using Interpolations
    
    # filter bank
    filter_hash = fink_filter_hash(1,8,nx=256,wd=2)
    
    function mnist_pad(im; θ=0.0)
        impad = zeros(Float64,128,128)
        impad[78:-1:51,51:78] = im'
        imbig = imresize(impad,(256,256))
        if θ != 0.0
            imrot = imrotate(imbig, θ, axes(imbig), Cubic(Throw(OnGrid())))
            imrot[findall(imrot .!= imrot)] .= 0.0
            return imrot
        end
        return imbig
    end
    
    function mnist_DHC(params)
        θ, x = params
        image    = mnist_pad(x[:,:], θ=θ)
        WST = DHC_compute(image, filter_hash)
        return WST
    end
    
end

mnist_DHC_out = @showprogress pmap(mnist_DHC, product(angle_array2,lst_test))
mnist_DHC_out = hcat(mnist_DHC_out...)

h5write("mnist_DHC_test_angles2.h5", "main/data", mnist_DHC_out)